# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [77]:
import os

data_files = ["/kaggle/input/ass1nlp/bigrams (2).txt"]


def load_ngrams(file_path):
    ngram_dict = {}
    with open(file_path, "r", encoding="latin-1") as f:
        for line in f:
            parts = line.strip().split()  
            if len(parts) > 1:
                freq = int(parts[0])  
                ngram = tuple(parts[1:])  
                ngram_dict[ngram] = freq
    return ngram_dict


ngram_data = {}
for file in data_files:
    file_path = file
    ngram_data[file] = load_ngrams(file_path)



In [78]:
from collections import defaultdict
bigrams = ngram_data["/kaggle/input/ass1nlp/bigrams (2).txt"]
def extract_unigrams_from_ngrams(bigrams, fivegrams):
    unigrams = defaultdict(int)

    for (w1, w2), freq in bigrams.items():
        unigrams[w1] += freq
        unigrams[w2] += freq
    return unigrams


unigrams = extract_unigrams_from_ngrams(bigrams, fivegrams)

In [79]:
def key_distance(c1: str, c2: str) -> float:
    coord = {
        'q': (0,0), 'w': (0,1), 'e': (0,2), 'r': (0,3), 't': (0,4), 'y': (0,5), 'u': (0,6), 'i': (0,7), 'o': (0,8), 'p': (0,9),
        'a': (1,0), 's': (1,1), 'd': (1,2), 'f': (1,3), 'g': (1,4), 'h': (1,5), 'j': (1,6), 'k': (1,7), 'l': (1,8),
        'z': (2,0), 'x': (2,1), 'c': (2,2), 'v': (2,3), 'b': (2,4), 'n': (2,5), 'm': (2,6)
    }
    if c1 not in coord or c2 not in coord:
        return float('inf') 
    
    x1, y1 = coord[c1]
    x2, y2 = coord[c2]
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5 

In [80]:
def get_keyboard_weight(c1: str, c2: str) -> float:
    dist = key_distance(c1, c2)
    return 1.0 / (1 + dist**2) if dist != 0 else 0.0



In [83]:
print(keyboard_proximity[('d','b')])

0.17


In [85]:
import itertools


keyboard_chars = "qwertyuiopasdfghjklzxcvbnm"

keyboard_proximity = {}
for c1 in keyboard_chars:
    for c2 in keyboard_chars:
        if c1 != c2:
            weight = get_keyboard_weight(c1, c2)
            if weight > 0.1:  
                keyboard_proximity[(c1, c2)] = round(weight, 2)




In [87]:
import re
from collections import defaultdict, Counter
import itertools



def edits1(word: str) -> set:
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word: str) -> set:
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))


from collections import defaultdict
import re
import math
class SpellCorrector:
    def __init__(self, unigrams, bigrams, beam_width, keyboard_proximity):
        self.unigrams = unigrams  
        self.bigrams = bigrams   
        self.beam_width = beam_width
        self.total_uni = sum(unigrams.values())
        
        self.keyboard_proximity = keyboard_proximity

    def P(self, word: str) -> float:
        return self.unigrams.get(word, 0) / self.total_uni

    def count_probs(self, bigram_key, word, context):
        bigram_freq = self.bigrams.get(bigram_key, 0)
        p_word = self.unigrams.get(word, 0)
        p_context = self.unigrams.get(context, 0) 
        if p_context > 0:
            return (bigram_freq/p_word) * (p_word/self.total_uni)/ (p_context/self.total_uni)       
        return self.P(word)

    
    def get_ngram_prob(self, context: tuple, word: str, index) -> float:
        prob = 0
        if index == 0:
            context = list(context)[0]
            bigram_key = (word,context)
            prob = self.count_probs(bigram_key, word, context)
        elif index > 0 and index < len(context):
            context1 = list(context)[index-1]
            bigram_key = (context1,word)
            prob = self.count_probs(bigram_key, word, context)
            context = list(context)[index]
            bigram_key = (word,context)
            prob += self.count_probs(bigram_key, word, context)
            prob /=2
        elif index == len(context):
            context = list(context)[index-1]
            bigram_key = (context,word)
            prob = self.count_probs(bigram_key, word, context)
                 
        return prob


    def score(self, candidate: str, context: list, word, index) -> float:

        key_weight = 1
        if len(candidate) == len(word):
            for c_cand, c_prev in zip(candidate, word):
                if c_cand == c_prev:
                    key_weight += 1
                else:
                    key_weight += self.keyboard_proximity.get((c_prev, c_cand), 0)
            key_weight -= 1
        key_weight /= len(word)  
        

        context_tuple = tuple(context)
        ngram_prob = self.get_ngram_prob(context_tuple, candidate, index)

        return (0.9 * ngram_prob + 0.1 * self.P(candidate)) * key_weight

    def correct(self, sentence: str) -> str:
        words = sentence.lower().split()
        beam = [([], 1.0)]  
        
        for idx in range(len(words)):
            current_word = words[idx]
            new_beam = []
            
            for corrected_seq, score_so_far in beam:
                candidates = (edits1(current_word) | {current_word} | edits2(current_word)) & self.unigrams.keys()
                
                if not candidates:
                    new_corrected = corrected_seq + [current_word]
                    new_beam.append((new_corrected, score_so_far))
                    continue
                
                for candidate in candidates:
                    context_for_score = corrected_seq + words[idx+1:]
                    candidate_score = self.score(candidate, context_for_score, current_word, idx)
                    
                    if candidate == current_word:
                        candidate_score *= 3  
                    
                    new_score = score_so_far * candidate_score
                    new_corrected = corrected_seq + [candidate]
                    new_beam.append((new_corrected, new_score))
            
    
            new_beam.sort(key=lambda x: -x[1])
            beam = new_beam[:self.beam_width]

        
        if not beam:
            return sentence
        
        best_hypothesis = max(beam, key=lambda x: x[1])
        return " ".join(best_hypothesis[0])





In [90]:
beam_width = 5

corrector = SpellCorrector(unigrams, bigrams, beam_width, keyboard_proximity)


print(corrector.correct("because you do n't	wamt")) 
print(corrector.correct("dking woman")) 

because you do n't want
dying woman


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

# N-gram Dataset Selection

I used a bigram dataset. This choice was made because bigrams allow us to model local word dependencies, improving the accuracy of word correction based on surrounding context. While higher-order n-grams (such as trigrams or five-grams) could provide additional context, they also introduce sparsity issues and require significantly larger datasets.

# Weight Assignments for Edit Probabilities

Edit distances are considered using the edits1 and edits2 functions, which generate candidate words with one or two character modifications. 

# Keyboard Proximity Weighting

I implemented a key_distance function to calculate the physical proximity between characters on a QWERTY keyboard. This was integrated into my scoring function to favor corrections that involve adjacent key presses, reflecting common typing mistakes. The weight for keyboard distance is Euclidian distance

# Beam Search Parameters

I used a beam search strategy to efficiently explore possible corrections while maintaining computational feasibility. The beam width was selected to balance performance and accuracy. A wider beam would consider more candidates, increasing correction accuracy, but at the cost of higher computational complexity. Conversely, a narrow beam would reduce processing time but might miss better corrections.
Because of this I chose beam width = 5.

# Scoring Function for Candidates

Scoring function combines:

-N-gram probability: The likelihood of a candidate word given its surrounding context.

-Unigram probability: The overall frequency of the word in the corpus.

-Keyboard proximity weight: The likelihood that a given word resulted from a nearby key press.

These factors were weighted to prioritize contextually appropriate words while accounting for common spelling mistakes.

# Test data set

The test dataset was created using a file containing five grams that were randomly modified to introduce errors. Despite achieving 36% accuracy, the model demonstrates acceptable logical and correct word substitutions.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [123]:
import random
def introduce_errors(sentence: str, error_prob=0.3) -> str:
    words = sentence.split()
    noisy_words = []
    for word in words:
        if random.random() < error_prob:
            noisy_words.append(random.choice(list(edits1(word)) + [word]))
        else:
            noisy_words.append(word)
    return " ".join(noisy_words)

In [124]:
test = {}
with open("/kaggle/input/ass1nlp/fivegrams (2).txt", "r", encoding="latin-1") as f:
    for i, line in enumerate(f):
        if i % 100000 == 0: 
            parts = line.strip().split()
            if len(parts) > 1:
                freq = int(parts[0])
                ngram = " ".join(parts[1:])  
                test[ngram] = introduce_errors(ngram)


In [126]:

def evaluate(corrector, test_sentences):
    total, correct = 0, 0
    for k,v in test_sentences.items():
        corrected_sentence = corrector.correct(v)
        print(f"corrected: {corrected_sentence}  true: {k}  error:{v}")
        if corrected_sentence == k:
            correct += 1
        total += 1
    return correct / total

In [127]:
accuracy = evaluate(corrector, test)
print(f"Accuracy: {accuracy:.2%}")


corrected: a have in the words  true: a babe in the woods  error:a bace in the fwoods
corrected: are likely to be quite  true: are likely to be quite  error:are likely to be qyuite
corrected: crisis that and to the  true: crisis that led to the  error:criois that led zo the
corrected: has been used in a  true: has been used in a  error:has been used int a
corrected: in of very good position  true: in a very good position  error:in m vnry good position
corrected: looked at the and smiled  true: looked at her and smiled  error:looked at her and smiled
corrected: of of the four or  true: one of the four or  error:onz ofl the four or
corrected: talk about the decline of  true: talk about the decline of  error:talk aboutm the decline ofi
corrected: the strength of the u.s  true: the strength of the u.s  error:kthe strength of thee u.s
corrected: to the findings of it  true: to the findings of a  error:to the findings of g
corrected: wife and to children and  true: wife and two children in  

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)